Biblioteki

In [30]:
import pandas as pd
from itertools import permutations

Słownik z punktami za dane miejsce w wyścigu

In [5]:
points_list = {1:25, 2:18, 3:15, 4:12, 5:10, 6:8,  7:6, 8:4, 9:2, 10:1, 11:0, 12:0}

Funkcja generujące kombinacje miejsc dająca mistrzostwo Ferrari

In [6]:
def ChampionshipOutcomes(fastest_lap=None):

    unique_combinations = set()
    filtered_combinations = []

    # Pętla generująca kombinacje
    for f1, f2, m1, m2 in permutations(range(1, 13), 4):
        # Unikalne pary
        ferrari_pair = tuple(sorted([f1, f2]))
        mclaren_pair = tuple(sorted([m1, m2]))
        combination = (ferrari_pair, mclaren_pair)

        # Dodajemy tylko unikalne pary
        if combination not in unique_combinations:
            unique_combinations.add(combination)

            # Ignorujemy pary Ferrari [1, 12]
            if ferrari_pair == (1, 12):
                continue

            ferrari_points = points_list[f1] + points_list[f2]
            mclaren_points = points_list[m1] + points_list[m2]

            # Dodanie punktu za najszybsze okrążenie
            if fastest_lap == "Ferrari":
                ferrari_points += 1
            elif fastest_lap == "McLaren":
                # McLaren ląduje na niepunktowanych pozycjach
                if not (m1 == 11 and m2 == 12):
                    mclaren_points += 1

            # Sprawdzenie czy Ferrari wygrało wyścig
            ferrari_has_first = 1 in (f1, f2)
            required_difference = 21 if ferrari_has_first else 22

            # Wybranie tych które dają mistrzostwo Ferrari
            if (ferrari_points - mclaren_points) >= required_difference:
                filtered_combinations.append({
                    "Places": [f1, f2, m1, m2],
                    "Ferrari points": ferrari_points,
                    "McLaren points": mclaren_points,
                    "Difference": ferrari_points - mclaren_points,
                })

    # Lista z kombinacjami miejsc                                   
    return filtered_combinations


Filtrowanie danych z niepotrzebnych kombinacji

In [34]:
def filter_combinations(outcomes):
    
    filtered_results = {}

    for outcome in outcomes:
        ferrari_pair = tuple(sorted(outcome["Places"][:2]))
        mclaren_leader = outcome["Places"][2]
        mclaren_follower = outcome["Places"][3]

        # Jeśli para Ferrari nie istnieje jeszcze w wynikach, dodajmy ją
        if ferrari_pair not in filtered_results:
            filtered_results[ferrari_pair] = []
        
        # Dodajemy nową kombinacje McLaren, jeśli nie jest gorsza niż już istniejąca
        is_redundant = False
        for existing in filtered_results[ferrari_pair]:
            existing_leader = existing["Places"][2]
            existing_follower = existing["Places"][3]

            # Jeśli zarówno pierwszy jak i drugi bolid są gorszi, oznacz jako zbędny
            if mclaren_leader >= existing_leader and mclaren_follower >= existing_follower:
                is_redundant = True
                break
        if not is_redundant:
            filtered_results[ferrari_pair].append(outcome)
    

    return [outcome for outcomes in filtered_results.values() for outcome in outcomes]

Funkcja tabeli

In [35]:
def generate_table(outcomes):
    # DataFrame
    df = pd.DataFrame(outcomes)

    # Rozbice tabeli "Places"
    df['Ferrari'] = df['Places'].apply(lambda x: f"{x[0]} & {x[1]}")
    df['McLaren'] = df['Places'].apply(lambda x: f"{x[2]} & {x[3]}")

    # Sortowanie według miejsc Ferrari
    df['Ferrari Pos 1'] = df['Places'].apply(lambda x: x[0])
    df['Ferrari Pos 2'] = df['Places'].apply(lambda x: x[1])
    
    # Usunięcie tabeli Places
    df = df.drop(columns='Places')

    # Sortowanie według punktow McLarena
    # df = df.sort_values(by='McLaren points', ascending=False)

    # Dodanie sortowania według miejsc Ferrari
    df = df.sort_values(by=['Ferrari Pos 1', 'Ferrari Pos 2'])
    
    # Ustalanie kolejności kolumn
    column_order = ['Ferrari', 'McLaren', 'Difference', 'Ferrari points', 'McLaren points']
    df = df[column_order]

    return df

Wyświetlenie Danych Możliwości

In [36]:
#Żadna ekpia nie ma najszybszego okrążenia
no_fastest_lap = ChampionshipOutcomes()
nobody_filtered_outcomes = filter_combinations(no_fastest_lap)
nobody_outcomes_table = generate_table(nobody_filtered_outcomes)

print(f"Liczba kombinacji: {len(no_fastest_lap)}")
print(f"Liczba kombinacji po filtracji {len(nobody_filtered_outcomes)}")
display(nobody_outcomes_table.style.hide(axis='index'))

Liczba kombinacji: 283
Liczba kombinacji po filtracji 36


Ferrari,McLaren,Difference,Ferrari points,McLaren points
1 & 2,3 & 7,22,43,21
1 & 2,4 & 5,21,43,22
1 & 3,2 & 10,21,40,19
1 & 3,4 & 7,22,40,18
1 & 3,5 & 6,22,40,18
1 & 4,3 & 10,21,37,16
1 & 4,5 & 7,21,37,16
1 & 5,4 & 9,21,35,14
1 & 5,6 & 7,21,35,14
1 & 6,4 & 11,21,33,12


In [38]:
# Ferrari zdobywa najszybsze okrążenie
ferrari_fastest_lap = ChampionshipOutcomes(fastest_lap="Ferrari")
ferrari_filtered_outcomes = filter_combinations(ferrari_fastest_lap)
ferrari_outcomes_table = generate_table(ferrari_filtered_outcomes)

print(f"Liczba kombinacji: {len(ferrari_fastest_lap)}")
print(f"Liczba kombinacji po filtracji {len(ferrari_filtered_outcomes)}")
display(ferrari_outcomes_table.style.hide(axis='index'))

Liczba kombinacji: 317
Liczba kombinacji po filtracji 37


Ferrari,McLaren,Difference,Ferrari points,McLaren points
1 & 2,3 & 6,21,44,23
1 & 2,4 & 5,22,44,22
1 & 3,2 & 9,21,41,20
1 & 3,4 & 6,21,41,20
1 & 4,3 & 9,21,38,17
1 & 4,5 & 7,22,38,16
1 & 5,3 & 11,21,36,15
1 & 5,4 & 9,22,36,14
1 & 5,6 & 7,22,36,14
1 & 6,4 & 10,21,34,13


In [41]:
# McLaren zdobywa najszybsze okrążenie
mclaren_fastest_lap = ChampionshipOutcomes(fastest_lap="McLaren")
mclaren_filtered_outcomes = filter_combinations(mclaren_fastest_lap)
mclaren_outcomes_table = generate_table(mclaren_filtered_outcomes)

print(f"Liczba kombinacji: {len(mclaren_fastest_lap)}")
print(f"Liczba kombinacji po filtracji {len(mclaren_filtered_outcomes)}")
display(mclaren_outcomes_table.style.hide(axis='index'))

Liczba kombinacji: 250
Liczba kombinacji po filtracji 32


Ferrari,McLaren,Difference,Ferrari points,McLaren points
1 & 2,3 & 7,21,43,22
1 & 2,4 & 6,22,43,21
1 & 3,2 & 11,21,40,19
1 & 3,4 & 7,21,40,19
1 & 3,5 & 6,21,40,19
1 & 4,3 & 11,21,37,16
1 & 4,5 & 8,22,37,15
1 & 4,6 & 7,22,37,15
1 & 5,4 & 10,21,35,14
1 & 5,6 & 8,22,35,13


Export danych do plików CSV

In [40]:
ferrari_outcomes_table.to_csv("Fastest lap Ferrari.csv", index=False)
nobody_outcomes_table.to_csv("Without fastest lap.csv", index=False)
mclaren_outcomes_table.to_csv("Fastest lap McLaren.csv", index=False)

Scalanie tabel z wynikami

In [42]:
# Ścieżki plików CSV
file_ferrari = "Fastest lap Ferrari.csv"
file_mclaren = "Fastest lap McLaren.csv"
file_without = "Without fastest lap.csv"

# Import danych CSV
fastest_ferrari = pd.read_csv(file_ferrari)
faster_mclaren = pd.read_csv(file_mclaren)
without_fastest = pd.read_csv(file_without)

# Dodanie kolumny "Scenario"
fastest_ferrari["Scenario"] = "Fastest for Ferrari"
faster_mclaren["Scenario"] = "Fastest for McLaren"
without_fastest["Scenario"] = "Without the fastest"

# Połączenie danych
all_results = pd.concat([fastest_ferrari, without_fastest, faster_mclaren])

# Export do pliku CSV
all_results.to_csv("Results table.csv", index=False)